In [ ]:
from PyQt5.QtWidgets import QFileDialog,QMainWindow,QDialog,QLabel,QTextEdit
from PyQt5.QtGui import QPixmap,QImage
from PyQt5.QtCore import QTimer
from itertools import combinations
import import_ipynb
import ui as UI
import cv2
import numpy as np
import glob
import os
class MainWindow_controller(QMainWindow):
    def __init__(self):
        super(MainWindow_controller, self).__init__() # in python3, super(Class, self).xxx = super().xxx   # 所以可簡寫成super().__init__()
        self.ui = UI.Ui_MainWindow()
        self.ui.setupUi(self)

        self.ui.pushButton.clicked.connect(self.openDirectoryDialog)
        self.ui.pushButton_2.clicked.connect(self.openImageLDialog)
        self.ui.pushButton_3.clicked.connect(self.openImageRDialog)
        self.ui.pushButton_4.clicked.connect(self.detectCorners)
        self.ui.pushButton_5.clicked.connect(self.findintrinsic)
        self.ui.spinBox.valueChanged.connect(self.update_which_image)
        self.ui.pushButton_6.clicked.connect(self.findextrinsic)
        self.ui.pushButton_7.clicked.connect(self.finddistortion)
        self.ui.pushButton_8.clicked.connect(self.showundistortion)
        # self.ui.text_edit.textChanged.connect(self.input_text)
        self.ui.pushButton_9.clicked.connect(self.showwordsonboard)
        self.ui.pushButton_10.clicked.connect(self.showwordvertically)
        self.ui.pushButton_11.clicked.connect(self.stereodisparitymap)
        self.ui.pushButton_12.clicked.connect(self.loadimage1)
        self.ui.pushButton_13.clicked.connect(self.loadimage2)
        self.ui.pushButton_14.clicked.connect(self.SIFTkeypoints)
        self.ui.pushButton_15.clicked.connect(self.matchedkeypoints)

        


    
    
    def openDirectoryDialog(self):          # 選擇目錄
        folder = QFileDialog.getExistingDirectory(self, 'Select Folder')    # QFileDialog.getExistingDirectory 是一個class method，不需要實例化 QFileDialog 物件就可以直接使用，這種方法通常用於調用靜態對話框。
        if folder:
            self.folder = folder        
            print(self.folder)
    
    def openImageLDialog(self):             # 選擇文件
        ImgL, _ = QFileDialog.getOpenFileName(self, 'Select imgL')   
        if ImgL:
            self.ImgL = ImgL        
            print(self.ImgL)
            

    def openImageRDialog(self):             # 選擇文件
        ImgR, _ = QFileDialog.getOpenFileName(self, 'Select imgR')    
        if ImgR:
            self.ImgR = ImgR        
            print(self.ImgR)


    def detectCorners(self):
        if self.folder:
            # 獲取文件夾中所有 bmp 文件
            # image_files = sorted(glob.glob(os.path.join(self.folder, "*.bmp")))[:15] # \1.bmp \10. \11. ... \2. \3.
            image_files = sorted(glob.glob(os.path.join(self.folder, "*.bmp")), key=lambda x: int(os.path.splitext(os.path.basename(x))[0]))[:15] # key=lambda x: 指定排序的依據, x: 代表每個檔案的路徑

            chessboard_size = (8, 11)   # 棋盤有 8x11 的內角點
            processed_images = []       # 儲存處理過的圖像
            img_points = []             # 用於保存所有圖像的二维圖像點
            object_list = []            # 用於保存所有object_points
            square_size = 20.0          # 每個方塊的邊長（unit ： mm）

            # # 定義物理點坐標（假設棋盤格在z=0的平面上）
            # object_points = np.zeros((chessboard_size[0] * chessboard_size[1], 3), np.float32)
            # object_points[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)
            # print(object_points[0])     # 三維 x軸向下，y軸向左
            # print(object_points[1])
            # object_points *= square_size    

            # 生成棋盤格並轉換到對應corner的座標點（假設在z=0的平面上）
            object_points = np.zeros((chessboard_size[0] * chessboard_size[1], 3), np.float32)
            grid = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]]     # 生成8*11網格
            swapped_xy = grid[[1, 0], :, :]                                 # x,y軸互換
            flipped_vertical = swapped_xy[:, ::-1, :]                       # 垂直翻轉
            object_points[:, :2] = flipped_vertical.T.reshape(-1, 2)
            object_points *= square_size        
            
            for file_path in image_files:
                image = cv2.imread(file_path)
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                print(file_path)
                
                ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)                               # 找到棋盤角點
                
                # # 替換第一個角點
                # new_first_points = np.array([[[500, 100]]])
                # corners[1] = new_first_points

                # # 替換最後一個角點
                # new_last_points = np.array([[[50, 50]]])
                # corners[-1] = new_last_points
              
                if ret:
                    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)                      # 設定停止條件 (maxIter=30, epsilon=0.001)      
                    corners = cv2.cornerSubPix(gray, corners, winSize=(5, 5), zeroZone=(-1, -1), criteria=criteria) # 進行角點的精確化 

                    cv2.drawChessboardCorners(image, chessboard_size, corners, ret)                                 # 繪製角點
                    img_points.append(corners)                                                      
                    object_list.append(object_points)                                         
                
                processed_images.append(image)              # 將處理後的圖像添加到列表

            self.img_gray = gray
            self.img_points = img_points
            self.object_list = object_list              

            self.showImagesInNewWindow(processed_images)    # 創建新窗口顯示圖片

    def showImagesInNewWindow(self, images):
        dialog = QDialog(self)                              # 創建新窗口
        dialog.setWindowTitle("Chessboard Images Slideshow")
        dialog.setGeometry(20, 20, 1400, 1000)

        label = QLabel(dialog)                              # 在 dialog 裡加入標籤，可顯示文字及圖片
        label.setGeometry(50, 50, 1300, 900)
        dialog.show()

        self.image_index = 0
        self.images = images
        self.timer = QTimer(dialog)                         # 在 dialog 裡加入QTimer                          
        def update_image():
            if self.image_index < len(self.images):
                img = self.images[self.image_index]
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # 圖片透過OpenCV讀進來的 預設為 BGR
                height, width, channels = img.shape
                bytesPerLine = 3 * width                    # 設定「每一行」的影像佔用位置數量，目前因為有 3 個 channel，因此是 3 * width
                qImg = QImage(img.data, width, height, bytesPerLine, QImage.Format_RGB888)          # 讀取圖片  #透過 PyQt5 的 QImage 方法將圖片轉換成 PyQt5 使用的圖片格式，再使用 QPixmap 讀取圖片
                label.setPixmap(QPixmap.fromImage(qImg).scaled(label.width(), label.height()))      # 加入圖片  #在 QLabel 裡加圖片，需使用 setPixmap() 方法加入圖片
                self.image_index += 1
            else:
                self.timer.stop()                           # 停止定時器
                QTimer.singleShot(2000, dialog.close)       # 2秒后關閉窗口

        self.timer.timeout.connect(update_image)            # 設定定時要啟用的 function
        self.timer.start(500)                               # 啟用定時器，設定間隔時間為 500 毫秒
        dialog.exec_()    


    def my_calibrateCamera(self, object_list, img_points):
        w = 2048                # (w,h): image size(2048, 2048)
        h = 2048
        # object_list, img_points 都是list
        # objectPoints = obj_points 是角點在棋盤上的 三维像素坐標 ， z=0
        # imagePoints = imagePoints 是角點在圖像上的 二维像素坐標，能由 findChessboardCorners 函數檢測得到
        ret, self.insintrinsic, self.distor, self.rvec, self.tvec=cv2.calibrateCamera (object_list, img_points, (w, h), None, None)   
        self.spinBoxindex = 0   # 設定spinBox的值

    def display_img(self, name, matrix):
        display_img = np.zeros((300, 800, 3), dtype=np.uint8)   # 創建一個空白圖像來顯示校正結果

        # 設定文字的格式
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.6
        color = (255, 255, 255)
        line_type = 1

        # 定義每行文字的間隔
        line_height = 25
        y_offset = 30

        # 將內參矩陣顯示在圖像上
        text = f"{name}:\n{matrix}"                             # f-string 格式化：字串前的 f 表示這是一個格式化字串，這樣可以在字串中直接插入變量值
        for i, line in enumerate(text.splitlines()):
            cv2.putText(display_img, line, (10, y_offset + i * line_height), font, font_scale, color, line_type)

        cv2.imshow(f"{name}", display_img)                      
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    def findintrinsic(self):
        self.my_calibrateCamera(self.object_list, self.img_points)
        print(self.insintrinsic)
        self.display_img("Intrinsic Matrix", self.insintrinsic)    

    def update_which_image(self, index):
        self.spinBoxindex = index -1

    def findextrinsic(self):
        R, _ = cv2.Rodrigues(self.rvec[self.spinBoxindex])  # 用cv2.Rodrigues將旋轉向量轉換為旋轉矩陣 
        T = self.tvec[self.spinBoxindex]                  
        extrinsic_matrix = np.hstack((R, T))                # 將 R 和 T 組合成 3x4 的外部參數矩陣
        print(extrinsic_matrix)
        self.display_img("Extrinsic Matrix", extrinsic_matrix)
    
    def finddistortion(self):
        print(self.distor)
        self.display_img("Distortion matrix", self.distor)

    def showundistortion(self):
        chessboard = cv2.cvtColor(self.img_gray, cv2.COLOR_GRAY2BGR)
        result_img = cv2.undistort(chessboard , self.insintrinsic, self.distor)      # Undistort the image by intrinsic matrix and distortion matrix
    
        width, height = 1000, 1000                              # 將旋轉向量轉換為旋轉矩陣
        # chessboard = cv2.resize(chessboard, (width, height))
        # result_img = cv2.resize(result_img, (width, height))

       
        # 顯示Distorted圖像的窗口
        cv2.namedWindow("Distorted Chessboard", cv2.WINDOW_NORMAL)
        cv2.resizeWindow("Distorted Chessboard", width, height)
        cv2.moveWindow("Distorted Chessboard", 100, 100)        # 設置窗口位置
        cv2.imshow("Distorted Chessboard", chessboard)

        # 顯示Undistorted圖像的窗口
        cv2.namedWindow("Undistorted Chessboard", cv2.WINDOW_NORMAL)
        cv2.resizeWindow("Undistorted Chessboard", width, height)
        cv2.moveWindow("Undistorted Chessboard", 1100, 100)     # 設置另一個窗口位置
        cv2.imshow("Undistorted Chessboard", result_img)

        cv2.waitKey(0)
        cv2.destroyAllWindows()

    # def input_text(self):
    #     # QTextEdit是一個class，需要先創建一個實例，才能對該實例進行操作
    #     text = self.ui.text_edit.toPlainText()         # 獲取輸入文字
    #     print("Text:", text)  
    #     char_list = list(text)                      # 將文字分成一個個字母，存入列表
    #     print("Character List:", char_list)  
    
    def showwordsonboard(self):

        if self.folder:
            image_files = sorted(glob.glob(os.path.join(self.folder, "*.bmp")))[:5]   
            chessboard_size = (8, 11)
            img_points = []             # 用於保存所有圖像的二维圖像點
            object_list = []            # 用於保存所有object_points  
            square_size = 20.0          # 每個方塊的邊長（unit ： mm）
            # 生成棋盤格並轉換到對應corner的座標點（假設在z=0的平面上）
            object_points = np.zeros((chessboard_size[0] * chessboard_size[1], 3), np.float32)
            grid = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]] #生成8*11網格
            swapped_xy = grid[[1, 0], :, :]                             #x,y軸互換
            flipped_vertical = swapped_xy[:, ::-1, :]                   #垂直翻轉
            object_points[:, :2] = flipped_vertical.T.reshape(-1, 2)
            object_points *= square_size            
                                
            Images = []                 # 用於保存所有圖像
            for file_path in image_files:
                image = cv2.imread(file_path)
                Images.append(image) 
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)             
                if ret:
                    img_points.append(corners)                                                      
                    object_list.append(object_points)
                
                                  
            _, ins, dist, rvecs, tvecs=cv2.calibrateCamera (object_list, img_points, (2048, 2048), None, None)   
            
            text = self.ui.textEdit.toPlainText()         # 獲取輸入文字記得要用self.ui...
            text_alphabet = list(text)
            print(text_alphabet)


            txt_file = os.path.join(self.folder, 'Q2_db', 'alphabet_db_onboard.txt')
            fs = cv2.FileStorage(txt_file, cv2.FILE_STORAGE_READ)  
            charPoints = list(map(lambda node: fs.getNode(node).mat(), text_alphabet))
            # charPoints = [fs.getNode(node).mat() for node in text_alphabet]
            # charPoints = fs.getNode(text_alphabet).mat()            # get the matrix
            

            # 使用字典來映射操作（更靈活和可維護）         
            def operation1(point):
                p2 = [[7, 5, 0],[7, 5, 0]]
                # for a, b in zip(p1, p2):
                #     result = a + b
                #     print(a)
                #     print(result) 
                result = [[a[i] + b[i] for i in range(len(point[0]))] for a, b in zip(point, p2)] # zip() 函數用於將可迭代的對象作順序組合
                return result
            def operation2(point):
                p2 = [[4, 5, 0],[4, 5, 0]]
                result = [[x[i] + y[i] for i in range(len(point[0]))] for x, y in zip(point, p2)] 
                return result
            def operation3(point):
                p2 = [[1, 5, 0],[1, 5, 0]]
                result = [[x[i] + y[i] for i in range(len(point[0]))] for x, y in zip(point, p2)] 
                return result
            def operation4(point):
                p2 = [[7, 2, 0],[7, 2, 0]]
                result = [[x[i] + y[i] for i in range(len(point[0]))] for x, y in zip(point, p2)] 
                return result
            def operation5(point):
                p2 = [[4, 2, 0],[4, 2, 0]]
                result = [[x[i] + y[i] for i in range(len(point[0]))] for x, y in zip(point, p2)] 
                return result
            def operation6(point):
                p2 = [[1, 2, 0],[1, 2, 0]]
                result = [[x[i] + y[i] for i in range(len(point[0]))] for x, y in zip(point, p2)] 
                return result
            
            # 創建操作字典
            operations = {
                0: operation1,
                1: operation2,
                2: operation3,
                3: operation4,
                4: operation5,
                5: operation6
            }

            object_points = []
            for i in range(len(charPoints)): 
                print(len(charPoints[i]))               
                for j in range(len(charPoints[i])):                    
                    temp = operations[i](charPoints[i][j])               
                    temp = np.array(temp)
                    temp = temp.reshape(-1, 3)                    
                    object_points.append(temp)   
           
            object_points = np.array(object_points)       # 轉換成numpy array
            print(object_points.shape)
            # 檢查你的輸入數據的shape和類型                  
            # print("shape:", object_points.shape)
            # print("type:", object_points.dtype)            
            reshaped_objectpoint = object_points.reshape(-1, 3)
            reshaped_objectpoint = reshaped_objectpoint.astype(np.float64)
            reshaped_objectpoint*= square_size              # 記得這裡的objectpoint也要*square_size
            
            processed_images = []
            #Project object 3D world coordinates on image 
            for i in range(len(Images)): 
                rvec = rvecs[i]  # 旋轉向量
                tvec = tvecs[i]  # 平移向量
                newCharPoints, jacobian  = cv2.projectPoints(reshaped_objectpoint,rvec, tvec, ins, dist)              
                
                # newCharPoints = newCharPoints.astype(int)
                newCharPoints.reshape(-1, 2)
                
                
                # 繪製每一對點之間的連線
                for j in range(0, len(newCharPoints) - 1, 2):  # 每次跳過2個點
                    # 獲取相鄰的點
                    point1 = newCharPoints[j]
                    point2 = newCharPoints[j + 1]
                    pt1 = tuple(np.round(point1[0]).astype(int))
                    pt2 = tuple(np.round(point2[0]).astype(int))
                    # 在圖像上畫線
                    cv2.line(Images[i], pt1, pt2, (0, 0, 255), 4)

                processed_images.append(Images[i])                  
            self.showImagesInNewWindow(processed_images)

    
    def showwordvertically(self):

        if self.folder:
            image_files = sorted(glob.glob(os.path.join(self.folder, "*.bmp")))[:5]   
            chessboard_size = (8, 11)
            img_points = []             # 用於保存所有圖像的二维圖像點
            object_list = []            # 用於保存所有object_points  
            square_size = 20.0          # 每個方塊的邊長（unit ： mm）
            # 生成棋盤格並轉換到對應corner的座標點（假設在z=0的平面上）
            object_points = np.zeros((chessboard_size[0] * chessboard_size[1], 3), np.float32)
            grid = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]] #生成8*11網格
            swapped_xy = grid[[1, 0], :, :]                             #x,y軸互換
            flipped_vertical = swapped_xy[:, ::-1, :]                   #垂直翻轉
            object_points[:, :2] = flipped_vertical.T.reshape(-1, 2)
            object_points *= square_size            
                                
            Images = []                 # 用於保存所有圖像
            for file_path in image_files:
                image = cv2.imread(file_path)
                Images.append(image) 
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)             
                if ret:
                    img_points.append(corners)                                                      
                    object_list.append(object_points)
                
                                  
            _, ins, dist, rvecs, tvecs=cv2.calibrateCamera (object_list, img_points, (2048, 2048), None, None)   
            
            text = self.ui.textEdit.toPlainText()         # 獲取輸入文字記得要用self.ui...
            text_alphabet = list(text)
            print(text_alphabet)


            txt_file = os.path.join(self.folder, 'Q2_db', 'alphabet_db_vertical.txt')
            fs = cv2.FileStorage(txt_file, cv2.FILE_STORAGE_READ)  
            charPoints = list(map(lambda node: fs.getNode(node).mat(), text_alphabet))
            # charPoints = [fs.getNode(node).mat() for node in text_alphabet]
            # charPoints = fs.getNode(text_alphabet).mat()            # get the matrix
            

            # 使用字典來映射操作（更靈活和可維護）         
            def operation1(point):
                p2 = [[7, 5, 0],[7, 5, 0]]
                # for a, b in zip(p1, p2):
                #     result = a + b
                #     print(a)
                #     print(result) 
                result = [[a[i] + b[i] for i in range(len(point[0]))] for a, b in zip(point, p2)] # zip() 函數用於將可迭代的對象作順序組合
                return result
            def operation2(point):
                p2 = [[4, 5, 0],[4, 5, 0]]
                result = [[x[i] + y[i] for i in range(len(point[0]))] for x, y in zip(point, p2)] 
                return result
            def operation3(point):
                p2 = [[1, 5, 0],[1, 5, 0]]
                result = [[x[i] + y[i] for i in range(len(point[0]))] for x, y in zip(point, p2)] 
                return result
            def operation4(point):
                p2 = [[7, 2, 0],[7, 2, 0]]
                result = [[x[i] + y[i] for i in range(len(point[0]))] for x, y in zip(point, p2)] 
                return result
            def operation5(point):
                p2 = [[4, 2, 0],[4, 2, 0]]
                result = [[x[i] + y[i] for i in range(len(point[0]))] for x, y in zip(point, p2)] 
                return result
            def operation6(point):
                p2 = [[1, 2, 0],[1, 2, 0]]
                result = [[x[i] + y[i] for i in range(len(point[0]))] for x, y in zip(point, p2)] 
                return result
            
            # 創建操作字典
            operations = {
                0: operation1,
                1: operation2,
                2: operation3,
                3: operation4,
                4: operation5,
                5: operation6
            }

            object_points = []
            for i in range(len(charPoints)): 
                print(len(charPoints[i]))               
                for j in range(len(charPoints[i])):                    
                    temp = operations[i](charPoints[i][j])               
                    temp = np.array(temp)
                    temp = temp.reshape(-1, 3)                    
                    object_points.append(temp)   
           
            object_points = np.array(object_points)       # 轉換成numpy array
            print(object_points.shape)
            # 檢查你的輸入數據的shape和類型                  
            # print("shape:", object_points.shape)
            # print("type:", object_points.dtype)            
            reshaped_objectpoint = object_points.reshape(-1, 3)
            reshaped_objectpoint = reshaped_objectpoint.astype(np.float64)
            reshaped_objectpoint*= square_size              # 記得這裡的objectpoint也要*square_size
            
            processed_images = []
            #Project object 3D world coordinates on image 
            for i in range(len(Images)): 
                rvec = rvecs[i]  # 旋轉向量
                tvec = tvecs[i]  # 平移向量
                newCharPoints, jacobian  = cv2.projectPoints(reshaped_objectpoint,rvec, tvec, ins, dist)              
                
                # newCharPoints = newCharPoints.astype(int)
                newCharPoints.reshape(-1, 2)
                
                
                # 繪製每一對點之間的連線
                for j in range(0, len(newCharPoints) - 1, 2):  # 每次跳過2個點
                    # 獲取相鄰的點
                    point1 = newCharPoints[j]
                    point2 = newCharPoints[j + 1]
                    pt1 = tuple(np.round(point1[0]).astype(int))
                    pt2 = tuple(np.round(point2[0]).astype(int))
                    # 在圖像上畫線
                    cv2.line(Images[i], pt1, pt2, (0, 0, 255), 4)

                processed_images.append(Images[i])                  
            self.showImagesInNewWindow(processed_images)

    def stereodisparitymap(self):
        imgL = cv2.imread(self.ImgL)
        grayL = cv2.cvtColor(imgL, cv2.COLOR_BGR2GRAY)    
        imgR = cv2.imread(self.ImgR)
        grayR = cv2.cvtColor(imgR, cv2.COLOR_BGR2GRAY)   
        # imgL = cv2.imread(self.ImgL, cv2.IMREAD_GRAYSCALE)
        # imgR = cv2.imread(self.ImgR, cv2.IMREAD_GRAYSCALE)

        # 創建 StereoBM 物件
        stereo = cv2.StereoBM_create(numDisparities=112, blockSize=21)  # numDisparities 參數設置視差搜索範圍，即每個像素最多會向右搜索的像素數。blockSize 控制在計算視差時，匹配區域的大小

        # 計算視差圖
        disparity = stereo.compute(grayL, grayR)
        
        # 將視差圖的值縮放到 0-255 的範圍
        disparity_normalized = cv2.normalize(disparity, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
        disparity_normalized = np.uint8(disparity_normalized)  # 轉換為 8 位圖像，便於顯示

        # 顯示Undistorted圖像的窗口
        cv2.namedWindow("imgL", cv2.WINDOW_NORMAL)
        cv2.resizeWindow("imgL", 800, 800)
        cv2.moveWindow("imgL", 100, 100)  # 設置另一個窗口位置
        cv2.imshow("imgL", imgL)

        # 顯示Undistorted圖像的窗口
        cv2.namedWindow("imgR", cv2.WINDOW_NORMAL)
        cv2.resizeWindow("imgR", 800, 800)
        cv2.moveWindow("imgR", 900, 100)  # 設置另一個窗口位置
        cv2.imshow("imgR", imgR)

        # 顯示Distorted圖像的窗口
        cv2.namedWindow("Disparity", cv2.WINDOW_NORMAL)
        cv2.resizeWindow("Disparity", 800, 800)
        cv2.moveWindow("Disparity", 1700, 100)  # 設置窗口位置
        cv2.imshow('Disparity', disparity_normalized)

        # 等待按鍵，然後關閉窗口
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    
    def loadimage1(self):
        #選擇文件
        Img1, _ = QFileDialog.getOpenFileName(self, 'Select Img1')   
        if Img1:
            self.Img1 = Img1        # 設置實例變數
            print(self.Img1)

    def loadimage2(self):
        #選擇文件
        Img2, _ = QFileDialog.getOpenFileName(self, 'Select Img2')   
        if Img2:
            self.Img2 = Img2        # 設置實例變數
            print(self.Img2)

    def SIFTkeypoints(self):      
        img = cv2.imread(self.Img1)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # 初始化 SIFT 檢測器
        sift = cv2.SIFT_create()

        # 檢測關鍵點和計算描述符
        keypoints, descriptors = sift.detectAndCompute(gray, None)

        # 在圖像上繪製關鍵點
        img_with_keypoints = cv2.drawKeypoints(gray, keypoints, None, color=(0,255,0))

        cv2.namedWindow("SIFT Keypoints", cv2.WINDOW_NORMAL)
        cv2.resizeWindow("SIFT Keypoints", 900, 900)
        cv2.moveWindow("SIFT Keypoints", 10, 10)
        cv2.imshow("SIFT Keypoints", img_with_keypoints)        
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    
    def matchedkeypoints(self):  
        img1 = cv2.imread(self.Img1)    
        img2 = cv2.imread(self.Img2)
        gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
        gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

        # 初始化 SIFT 檢測器
        sift = cv2.SIFT_create()

        # 檢測關鍵點和計算描述符
        keypoints1, descriptors1 = sift.detectAndCompute(gray1, None)
        keypoints2, descriptors2 = sift.detectAndCompute(gray2, None)

        # 使用 KNN 進行匹配（找出兩個最近的點）
        matches = cv2.BFMatcher().knnMatch(descriptors1, descriptors2, k=2)

        # 篩選出優質的匹配
        good_matches = []
        for m, n in matches:
            if m.distance < 0.75 * n.distance:
                good_matches.append(m)

        # 繪製優質匹配結果
        img_matches = cv2.drawMatches(gray1, keypoints1, gray2, keypoints2, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

        # 顯示結果
        cv2.namedWindow("Good Matched Keypoints", cv2.WINDOW_NORMAL)
        cv2.resizeWindow("Good Matched Keypoints", 1800, 900)
        cv2.moveWindow("Good Matched Keypoints", 10, 10)
        cv2.imshow("Good Matched Keypoints", img_matches)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
        

       


    
    
            
    
   